25년 1월 12일부터 매일 하루 1회 돌리기

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

In [3]:
# 폴더 내의 모든 CSV 파일 가져오기
item_files = [file for file in os.listdir('./data') if file.endswith('.csv')]

# CSV 파일을 딕셔너리로 저장
dataframes_dict = {}
for csv_file in item_files:
    file_path = os.path.join('./data', csv_file)  # 전체 파일 경로
    df = pd.read_csv(file_path)  # CSV 파일 읽기
    csv_file = re.sub('.csv', '', csv_file)  # .csv 확장자 제거
    dataframes_dict[csv_file] = df  # 파일 이름을 키로 사용

# # 딕셔너리 확인
# for file_name, df in dataframes_dict.items():
#     print(f"File: {file_name}")
#     print(df.head())  # 각 데이터프레임의 상위 5개 행 출력


In [4]:
# api로 하루 전 날짜의 데이터 가져오기

import requests
import json
from lostark_api_token import Token, total_item_df

headers = {
    'accept' : 'application/json',
    'authorization' : Token
}

# 아이템 코드로 아이템 정보 가져오기
def get_item_info(item_code):
    url = f'https://developer-lostark.game.onstove.com/markets/items/{item_code}'
    response = requests.get(url, headers=headers)
    jsonObject = response.json()
    return jsonObject

# 아이템 정보를 데이터프레임으로 변환
def item_info_yesterday(item_info):
    name = item_info[0]['Name']
    stats = item_info[0]['Stats'][1]
    
    # 특수 문자 제거 및 띄어쓰기 제거
    safe_item_name = re.sub(r'[\\/:*?"<>|[\]]', '', name)  # 특수 문자를 제거
    safe_item_name = safe_item_name.replace(' ', '')  # 띄어쓰기 제거
    stats['item_name'] = name  # 아이템 이름 추가
    return stats

In [5]:
# 아이템 코드로 아이템 정보 갱신하기

for item_code in total_item_df['item_code']:
    item_info = get_item_info(item_code)
    item_info_1day = item_info_yesterday(item_info)
    safe_item_name = re.sub(r'[\\/:*?"<>|[\]]', '', item_info_1day['item_name'])  # 특수 문자를 제거
    safe_item_name = safe_item_name.replace(' ', '')  # 띄어쓰기 제거
    new_df = pd.concat([dataframes_dict[safe_item_name], pd.DataFrame([item_info_1day])])
    dataframes_dict[safe_item_name] = new_df
    dataframes_dict[safe_item_name].to_csv(f"./data/{safe_item_name}.csv", index=False, encoding='utf-8-sig')